#Criando os nomes

In [ ]:
import pandas as pd

In [ ]:
nomes_m = pd.read_json("https://servicodados.ibge.gov.br/api/v1/censos/nomes/ranking?qtd=200&sexo=m")

In [ ]:
nomes_f = pd.read_json("https://servicodados.ibge.gov.br/api/v1/censos/nomes/ranking?qtd=200&sexo=f")

In [ ]:
nomes_m.head(200)

In [ ]:
print("Quantidade de nomes: " + str(len(nomes_f) + len(nomes_m)))

In [ ]:
frames = [nomes_f, nomes_m]

In [ ]:
frames

In [ ]:
nomes = pd.concat(frames)['nome'].to_frame()

In [ ]:
nomes.sample(5)

#Incluindo ID dos Alunos

In [ ]:
import numpy as np
np.random.seed(123)

In [ ]:
total_alunos = len(nomes)
total_alunos

In [ ]:
nomes['id_aluno'] = np.random.permutation(total_alunos) + 1

In [ ]:
nomes.sample(5)

In [ ]:
dominios = ['@dominiodoemail.com.br', '@servicodoemail.com']
nomes['dominio'] = np.random.choice(dominios, total_alunos)

In [ ]:
nomes

In [ ]:
nomes['email'] = nomes.nome.str.cat(nomes.dominio).str.lower()

In [ ]:
nomes

#Criando a tabela Cursos

In [ ]:
!pip3 install html5lib
!pip3 install lxml


In [ ]:
import html5lib

In [ ]:
url = 'http://tabela-cursos.herokuapp.com/index.html'
cursos = pd.read_html(url)
cursos

In [ ]:
type(cursos)

In [ ]:
cursos = cursos[0]

In [ ]:
type(cursos)

In [ ]:
cursos.head()

#**Alterando o index de cursos**

In [ ]:
cursos = cursos.rename(columns={'Nome do curso' : 'nome_do_curso'})


In [ ]:
cursos.head(2)

In [ ]:
cursos['id'] = cursos.index + 1

In [ ]:
cursos.head()

In [ ]:
cursos = cursos.set_index('id')

In [ ]:
cursos.head(5)

#**Matriculando os alunos nos cursos**

In [ ]:
nomes.sample(5)

In [ ]:
nomes['matriculas'] = np.ceil(np.random.exponential(size=total_alunos)*1.5).astype(int)

In [ ]:
nomes.sample(5)

In [ ]:
nomes.matriculas.describe()

In [ ]:
import seaborn as sns

In [ ]:
sns.distplot(nomes.matriculas)

In [ ]:
nomes.matriculas.value_counts()

#**Selecionando Cursos**

In [ ]:
nomes.sample(3)

In [ ]:
todas_matriculas = []
x = np.random.rand(20)
prob = x / sum(x)

In [ ]:
for index, row in nomes.iterrows():
  id = row.id_aluno
  matriculas = row.matriculas
  for i in range(matriculas):
    mat = [id, np.random.choice(cursos.index, p = prob)]
    todas_matriculas.append(mat)

matriculas = pd.DataFrame(todas_matriculas, columns = ['id_aluno', 'id_curso'])

In [ ]:
matriculas.head()

In [ ]:
matriculas_por_curso = matriculas.groupby('id_curso').count().join(cursos['nome_do_curso']).rename(columns={'id_aluno' : 'quantidade_de_alunos'})

In [ ]:
nomes.sample(3)

In [ ]:
cursos.head()

In [ ]:
matriculas.head()

In [ ]:
matriculas_por_curso

In [ ]:
matriculas_por_curso.to_csv('matriculas_por_curso.csv', index=False)

In [ ]:
pd.read_csv('matriculas_por_curso.csv')

In [ ]:
matriculas_json = matriculas_por_curso.to_json()

In [ ]:
matriculas_json

In [ ]:
matriculas_html = matriculas_por_curso.to_html()

In [ ]:
print(matriculas_html)

#**Criando o banco SQL**

In [ ]:
!pip install sqlalchemy

In [ ]:
from sqlalchemy import create_engine, MetaData, Table

In [ ]:
engine = create_engine('sqlite:///:memory')

In [ ]:
engine

In [ ]:
type(engine)

In [ ]:
matriculas_por_curso.to_sql('matriculas', engine)

In [ ]:
print(engine.table_names())

#**Buscando do banco SQL**

In [ ]:
query = 'select * from matriculas where quantidade_de_alunos < 20'

In [ ]:
pd.read_sql(query, engine)

In [ ]:
muitas_matriculas = pd.read_sql_table('matriculas', engine, columns=['nome_do_curso', 'quantidade_de_alunos'])

In [ ]:
muitas_matriculas

In [ ]:
muitas_matriculas = muitas_matriculas.query('quantidade_de_alunos > 80')

In [ ]:
muitas_matriculas

#**Escrevendo no banco**

In [ ]:
muitas_matriculas.to_sql('muitas_matriculas', con=engine)

In [ ]:
print(engine.table_names())

#**Nomes dos alunos e alunas da próxima turma**

In [ ]:
matriculas_por_curso.head(20)

In [ ]:
matriculas

In [ ]:
id_curso = 15
proxima_turma = matriculas.query("id_curso == {}".format(id_curso))
proxima_turma

In [ ]:
nomes.sample(3)

In [ ]:
proxima_turma.set_index('id_aluno').join(nomes.set_index('id_aluno'))

In [ ]:
proxima_turma.set_index('id_aluno').join(nomes.set_index('id_aluno'))['nome']

In [ ]:
proxima_turma = proxima_turma.set_index('id_aluno').join(nomes.set_index('id_aluno'))['nome'].to_frame()
proxima_turma

In [ ]:
nome_curso = cursos.loc[id_curso]
nome_curso

In [ ]:
nome_curso = nome_curso.nome_do_curso
nome_curso

In [ ]:
proxima_turma = proxima_turma.rename(columns = {'nome' : 'Alunos do curso {}'.format(nome_curso)})

In [ ]:
proxima_turma.sample(5)

#**Excel**

In [ ]:
proxima_turma.to_excel('proxima_turma.xlsx', index = False)